In [97]:
# Análisis EDA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sqlite3

#CONEXIÓN
conexion = sqlite3.connect("datos_sensores.db")

#CÓDIGO SQL PARA HACER LA CONSULTA
query = """
SELECT
datos_basicos.tamaño,
datos_basicos.categoria,
orion.sensor_orion,
vega.sensor_vega,
polaris.sensor_polaris,
antares.sensor_antares,
--clasificacion.etiqueta,

CASE
	WHEN clasificacion.etiqueta = "Positivo" THEN 1
  ELSE 0
END AS etiqueta_numerica

FROM datos_basicos

INNER JOIN orion ON datos_basicos.id = orion.id
INNER JOIN vega ON datos_basicos.id = vega.id
INNER JOIN polaris ON datos_basicos.id = polaris.id
INNER JOIN antares ON datos_basicos.id = antares.id
INNER JOIN clasificacion ON datos_basicos.id = clasificacion.id
AND clasificacion.etiqueta NOTNULL
 """
query_sin_etiqueta = """
SELECT
datos_basicos.tamaño,
datos_basicos.categoria,
orion.sensor_orion,
vega.sensor_vega,
polaris.sensor_polaris,
antares.sensor_antares,
--clasificacion.etiqueta,

CASE
	WHEN clasificacion.etiqueta = "Positivo" THEN 1
	ELSE clasificacion.etiqueta = clasificacion.etiqueta

END AS etiqueta_numerica

FROM datos_basicos

INNER JOIN orion ON datos_basicos.id = orion.id
INNER JOIN vega ON datos_basicos.id = vega.id
INNER JOIN polaris ON datos_basicos.id = polaris.id
INNER JOIN antares ON datos_basicos.id = antares.id
INNER JOIN clasificacion ON datos_basicos.id = clasificacion.id
AND clasificacion.etiqueta IS NULL
"""

In [98]:
dataset = pd.read_sql_query(query,conexion)
dataset

,tamaño,categoria,sensor_orion,sensor_vega,sensor_polaris,sensor_antares,etiqueta_numerica
0,0.02,3,19.77,42.15,35.36,-62.73,1
1,0.11,3,-23.51,-20.64,69.78,-38.02,1
2,0.85,1,86.63,75.26,30.82,-71.62,0
3,0.53,1,-44.91,4.11,71.92,27.17,0
4,0.14,2,-66.34,-48.04,-2.91,-74.93,0
...,...,...,...,...,...,...,...
299995,0.30,2,-81.98,20.64,8.07,40.18,1
299996,0.21,3,-2.74,-81.87,-15.87,40.64,1
299997,0.12,2,-13.86,-27.85,63.55,19.37,0
299998,0.09,1,-3.87,-69.29,86.51,-15.73,1


In [99]:
dataset_sin_etiquetas = pd.read_sql(query_sin_etiqueta , conexion)
dataset_sin_etiquetas

,tamaño,categoria,sensor_orion,sensor_vega,sensor_polaris,sensor_antares,etiqueta_numerica
0,0.10,2,-91.99,90.35,12.85,45.19,None
1,0.93,2,2.98,14.86,-22.14,45.21,None
2,0.86,3,-22.53,10.25,74.24,1.93,None
3,0.86,1,24.14,-31.35,-67.50,29.67,None
4,0.92,3,16.14,98.17,-14.76,10.59,None
...,...,...,...,...,...,...,...
95,0.61,3,75.72,-65.18,75.00,-33.67,None
96,0.08,1,-3.33,-28.75,-97.28,30.82,None
97,0.41,3,-48.04,-23.84,32.14,-46.10,None
98,0.71,1,15.46,-39.97,45.71,64.84,None


In [105]:
# Preprocesamiento de datos
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
#Módulo de métricas para medir que tan bien está
from sklearn.metrics import accuracy_score, classification_report

X = dataset.drop('etiqueta_numerica',axis=1) #Variables predictoras
y = dataset['etiqueta_numerica'] #Variable target

# Dividir datos en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.005)

In [106]:
# 3. Entrenar modelos
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier()

# Entrenar modelo
rf_model.fit(X_train, y_train)

RandomForestClassifier()

In [107]:
#Comprobar funcionamiento del modelo
predicciones = rf_model.predict(X_test)
X_test

,tamaño,categoria,sensor_orion,sensor_vega,sensor_polaris,sensor_antares
183290,0.92,1,-70.22,95.72,67.20,28.47
40996,0.24,1,-54.55,-15.84,-80.67,58.96
297529,0.08,2,12.69,-72.80,-1.91,89.41
82581,0.79,3,23.67,-75.76,21.64,-68.27
110507,0.39,3,-84.87,12.42,-14.26,99.14
...,...,...,...,...,...,...
275611,0.99,1,40.40,39.57,-24.87,-2.03
41509,0.31,3,-96.52,-23.17,4.17,-33.44
295347,0.16,1,-13.41,-78.86,51.52,0.87
265401,0.00,3,-13.20,39.02,1.75,-88.58


In [108]:
predicciones

array([0, 0, 0, ..., 1, 1, 1])

In [109]:
#Nivel de precisión del modelo
accuracy = accuracy_score (y_test,predicciones)
print(f"Precisión del modelo: {accuracy:.2f}")

Precisión del modelo: 0.91


In [110]:
reporte = classification_report (y_test,predicciones)
print(reporte)

              precision    recall  f1-score   support

           0       0.96      0.86      0.90       780
           1       0.86      0.96      0.91       720

    accuracy                           0.91      1500
   macro avg       0.91      0.91      0.91      1500
weighted avg       0.91      0.91      0.91      1500



**CLASIFICACIÓN DE LOS DATOS SIN ETIQUETA**

In [111]:
#Dataset de los datos sin etiqueta
x1 = dataset_sin_etiquetas.drop('etiqueta_numerica',axis=1) #Variables predictoras
y1 = rf_model.predict(x1)

In [112]:
#Convertir los datos a las etiquetas requeridas
dataset_salida = []
for predicciones in y1:
    if predicciones == 1:
        dataset_salida.append("positivo")
    else:
        dataset_salida.append("negativo")

print(dataset_salida)

['negativo', 'negativo', 'positivo', 'positivo', 'positivo', 'positivo', 'negativo', 'negativo', 'positivo', 'positivo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'positivo', 'positivo', 'negativo', 'negativo', 'negativo', 'positivo', 'positivo', 'positivo', 'negativo', 'negativo', 'positivo', 'negativo', 'positivo', 'negativo', 'negativo', 'negativo', 'positivo', 'negativo', 'negativo', 'positivo', 'negativo', 'negativo', 'positivo', 'negativo', 'negativo', 'positivo', 'negativo', 'negativo', 'positivo', 'positivo', 'positivo', 'negativo', 'negativo', 'positivo', 'positivo', 'negativo', 'negativo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'negativo', 'positivo', 'positivo', 'negativo', 'negativo', 'positivo', 'positivo', 'negativo', 'negativo', 'positivo', 'positivo', 'negativo', 'positivo', 'negativo', 'positivo', 'negativo', 'positivo', 'positivo', 'negativo', 'positivo', 'positivo', 'positivo', 'negativo', 'negativo', 'negativo', 'negativo', 'ne

In [132]:
#Guardar predicciones
query_id = """ SELECT
datos_basicos.id
FROM datos_basicos

INNER JOIN clasificacion ON datos_basicos.id = clasificacion.id

AND clasificacion.etiqueta IS NULL;
 """
dataset_id = pd.read_sql_query(query_id,conexion)
dataset_id

df_salida = pd.DataFrame(dataset_salida, columns=['etiqueta'])
df_salida['id'] = dataset_id['id']
df_salida.to_csv('predicciones.csv', index=False)
df_salida

,etiqueta,id
0,negativo,300001
1,negativo,300002
2,positivo,300003
3,positivo,300004
4,positivo,300005
...,...,...
95,positivo,300096
96,positivo,300097
97,positivo,300098
98,positivo,300099
